In [61]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [62]:
# --- Helper Function to Get Difference Table and Coefficients (MODIFIED) ---
def EvenDifference(points, condition=1):
    """
    Calculates the difference table and extracts coefficients.

    Args:
        points (list of tuples): A list of (x, y) data points.
        condition (int): 1 for Forward (top diagonal), 0 for Backward (bottom diagonal).

    Returns:
        tuple: A tuple containing:
            - pandas.DataFrame: The full difference table.
            - list: The list of extracted coefficients.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # --- NEW: Format the output DataFrame ---
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        data[f'Order {j}'] = diff_calc_table[:, j]
    
    df = pd.DataFrame(data)

    # Extract coefficients (logic is unchanged)
    if condition == 1:
        coefficients = diff_calc_table[0, :].tolist()
    else:
        coeffs = []
        for j in range(n):
            coeffs.append(diff_calc_table[n-1-j, j])
        coefficients = coeffs

    return df, coefficients

In [63]:
scores = [(0.0, 1.0), (1.0, 4.0), (2.0, 13.0), (3.0, 34.0), (4.0, 73.0)]

df, coeffs = EvenDifference(scores, condition = 1)

In [64]:
df

,x_i,y_i,Order 1,Order 2,Order 3,Order 4
0,0.0,1.0,3.0,6.0,6.0,0.0
1,1.0,4.0,9.0,12.0,6.0,NaN
2,2.0,13.0,21.0,18.0,NaN,NaN
3,3.0,34.0,39.0,NaN,NaN,NaN
4,4.0,73.0,NaN,NaN,NaN,NaN


In [65]:
coeffs

[1.0, 3.0, 6.0, 6.0, 0.0]

## Newton Interpolation for Even-distributed Node

## Algorithm


In [66]:
def NewtonInterpolation(points, condition=1):
    """
    Constructs the Newton interpolation polynomial and returns the steps
    and final coefficients as polished Pandas DataFrames.
    """
    n = len(points) - 1
    if n < 0:
        return pd.DataFrame(), pd.DataFrame()

    diff_table, diff_coeffs = EvenDifference(points, condition=condition)
    print("--- Generated Difference Table ---")
    print(diff_table.to_string())
    print("-" * 30)

    # --- NEW: Polished data structures for the output DataFrames ---
    steps_data = []
    N_coeffs_var = np.zeros(n + 1, dtype=float)
    B_coeffs_prev = np.array([1.0])

    for i in range(n + 1):
        D_i = diff_coeffs[i] / math.factorial(i)
        if i == 0:
            B_coeffs = np.array([1.0], dtype=float)
        else:
            k = i - 1
            varB = np.concatenate(([0.0], B_coeffs_prev))
            kB = np.concatenate((k * B_coeffs_prev, [0.0]))
            B_coeffs = varB - kB if condition == 1 else varB + kB
        
        B_coeffs_prev = B_coeffs.copy()
        Ni_coeffs = D_i * B_coeffs
        N_coeffs_var[:len(Ni_coeffs)] += Ni_coeffs

        # Append the polished row of data for the steps DataFrame
        steps_data.append({
            'i': i,
            'Diff Coeff': diff_coeffs[i],
            'D_i': D_i,
            'B_i Coeffs': B_coeffs.tolist(),
            'N_i Coeffs': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree': np.arange(n + 1),
        'Coeff': N_coeffs_var
    })

    return step_pd, coeff_pd

## Result

In [67]:
scores = [(0.0, 1.0), (1.0, 4.0), (2.0, 13.0), (3.0, 34.0), (4.0, 73.0)]

step_df, final_coeff_df = NewtonInterpolation(input_points, condition=1)

--- Generated Difference Table ---
   x_i   y_i  Order 1  Order 2  Order 3  Order 4
0  0.0   1.0      3.0      6.0      6.0      0.0
1  1.0   4.0      9.0     12.0      6.0      NaN
2  2.0  13.0     21.0     18.0      NaN      NaN
3  3.0  34.0     39.0      NaN      NaN      NaN
4  4.0  73.0      NaN      NaN      NaN      NaN
------------------------------


In [68]:
step_df.style

,i,Diff Coeff,D_i,B_i Coeffs,N_i Coeffs
0,0,1.000000,1.000000,[1.0],[1.0]
1,1,3.000000,3.000000,"[0.0, 1.0]","[0.0, 3.0]"
2,2,6.000000,3.000000,"[0.0, -1.0, 1.0]","[0.0, -3.0, 3.0]"
3,3,6.000000,1.000000,"[0.0, 2.0, -3.0, 1.0]","[0.0, 2.0, -3.0, 1.0]"
4,4,0.000000,0.000000,"[0.0, -6.0, 11.0, -6.0, 1.0]","[0.0, -0.0, 0.0, -0.0, 0.0]"


In [69]:
final_coeff_df.style

,Degree,Coeff
0,0,1.000000
1,1,2.000000
2,2,0.000000
3,3,1.000000
4,4,0.000000
